In [1]:
import os
import h5py
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import tensorflow as tf
from tensorflow.python.keras.models import Input, Sequential, Model
from tensorflow.python.keras.layers import Dense, Dropout, LSTM, Activation, Add
from tensorflow.python.keras.utils import Sequence
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import adam, SGD
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.keras.utils import plot_model

In [3]:
from modis_utils.misc import restore_data, cache_data

In [4]:
data = restore_data('cache/boundary_vectors_ALL.h5')

In [5]:
train_boundary_vectors = data[0]
val_boundary_vectors = data[1]
test_boundary_vectors = data[2]

In [6]:
def transform(data, scaler):
    old_shape = data.shape
    data = data.reshape(old_shape[0], -1)
    data = scaler.transform(data.astype(np.float))
    #return data.reshape(old_shape)
    return data

In [7]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_boundary_vectors.reshape(train_boundary_vectors.shape[0], -1))
train_boundary_vectors_scale = transform(train_boundary_vectors, scaler)
val_boundary_vectors_scale = transform(val_boundary_vectors, scaler)
test_boundary_vectors_scale = transform(test_boundary_vectors, scaler)

In [8]:
def create_dataset(boundary_vectors_scale, timesteps):
    data_X = []
    data_Y = []
    for i in range(len(boundary_vectors_scale) - timesteps):
        data_x = boundary_vectors_scale[i:(i+timesteps)]
        data_y = boundary_vectors_scale[i + timesteps]
        data_X.append(data_x)
        data_Y.append(data_y)
    return np.asarray(data_X), np.asarray(data_Y)

In [9]:
timesteps = 50
train_X, train_Y = create_dataset(train_boundary_vectors_scale, timesteps)
val_X, val_Y = create_dataset(np.concatenate(
    [train_boundary_vectors_scale[-timesteps:], val_boundary_vectors_scale]),
                              timesteps)
test_X, test_Y = create_dataset(np.concatenate(
    [val_boundary_vectors_scale[-timesteps:], test_boundary_vectors_scale]),
                                timesteps)

In [10]:
val_X.shape, val_Y.shape

((138, 50, 2048), (138, 2048))

In [11]:
test_X.shape, test_Y.shape

((138, 50, 2048), (138, 2048))

In [12]:
train_X.shape, train_Y.shape

((388, 50, 2048), (388, 2048))

In [13]:
def custom_loss_tf(y_true, y_pred):
    square_error = (y_true - y_pred)**2
    return tf.reduce_mean(tf.sqrt(tf.reduce_mean(square_error, axis=1)))
  
def custom_loss(y_true, y_pred):
    square_error = (y_true - y_pred)**2
    x = np.sqrt(np.mean(square_error, axis=1))
    print(x.shape)
    return np.mean(x)

In [14]:
def plot_model_(model):
    plot_model(model, to_file='model.png', show_shapes=True)
    model_plot = misc.imread('model.png')
    plt.imshow(model_plot)

In [15]:
def create_model(timesteps):
    inputs = Input(shape=(timesteps, 2048))
    x = LSTM(1024, return_sequences=True, activation='tanh')(inputs)
    x = LSTM(1024, return_sequences=True, activation='sigmoid')(x)
    predict = LSTM(2048, return_sequences=False, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=predict)
    return model

In [16]:
K.clear_session()
model = create_model(timesteps)

In [17]:
model.compile(loss=custom_loss_tf, optimizer=adam(lr=0.001))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 2048)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 50, 1024)          12587008  
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (None, 2048)              25174016  
Total params: 46,153,728
Trainable params: 46,153,728
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.load_weights('boundary_predict_1.h5')

In [19]:
!nvidia-smi

Mon Mar 25 22:04:31 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.183      Driver Version: 384.183      CUDA Version: 9.0      |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 960M    Off  | 00000000:02:00.0 Off |                  N/A |
| N/A   50C    P0    N/A /  N/A |   3778MiB /  4044MiB |     30%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [20]:
predict_dir = 'predict_boundary_samples'
if not os.path.exists(predict_dir):
    os.makedirs(predict_dir)

In [21]:
def inference(X, Y, model):
    test_inference = []
    test_loss = []
    for i in range(len(X)):
        data_X = X[i : i+1]
        data_Y = Y[i : i+1]
        try:
            predict = model.predict(data_X)
            loss = custom_loss(data_Y, predict)
            test_inference.append(predict)
            test_loss.append(loss)
        except:
            print(i)
            exit()
    test_inference = np.vstack(test_inference)
    test_loss = np.vstack(test_loss)
    return test_inference, test_loss

In [22]:
test_inference, test_loss = inference(test_X, test_Y, model)

(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)


In [23]:
np.mean(test_loss)

0.13093605241611972

In [24]:
train_inference, train_loss = inference(train_X, train_Y, model)
val_inference, val_loss = inference(val_X, val_Y, model)

(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)


In [26]:
cache_data((train_inference, val_inference, test_inference),
           os.path.join(predict_dir, 'inference_boundary_sample.dat'))
cache_data((train_loss, val_loss, test_loss),
           os.path.join(predict_dir, 'inference_boundary_loss.dat'))

In [28]:
import pickle
with open('scaler.dat', 'wb') as f:
    pickle.dump(scaler, f)